In [27]:
import pandas as pd
import requests as rq
import time
import zipfile
import os
from apikey_binance import API_KEY

BASE_URL = 'https://fapi.binance.com'
DATE_FROM = pd.Timestamp('2021-01-01')
DATE_TILL = pd.Timestamp('2022-01-31')
SYMBOLS = ['BTCUSDT', 'ETHUSDT']
FROM_IDS = [352562763, 205368939]
PATH = '../../ticks/binance/'
DEFAULT_WAIT = 0.5

In [32]:
def read_file(symbol, zip_name, missing):
    filename = PATH + f'{symbol}/{zip_name}'
    # print(f'reading \'{filename}\'... ')
    extention = filename[-4:]
    if extention == '.zip':
        csv_name = zip_name.replace('.zip', '.csv')
        zf = zipfile.ZipFile(filename) 
        df = pd.read_csv(zf.open(csv_name), header=None)
    elif extention == '.csv':        
        df = pd.read_csv(filename)
    else:
        print('Unsupported extention')
        return None

    no_idx = len(df.columns) == 6
    if no_idx:
        df.columns = ['id', 'price', 'size', 'quote_size', 'timestamp', 'is_sell']
    else:
        df.columns = ['idx', 'id', 'price', 'size', 'quote_size', 'timestamp', 'is_sell']

    def count_missng(row):
        id_till = row['id'] - 1
        id_from = id_till - row['delta_id'] + 1
        time_delta = (row['timestamp'] - row['prev_time']).total_seconds()
        missing.append({'from': id_from, 'till': id_till, 'time_delta': time_delta})
        if (time_delta > 3):
            print(f'{zip_name}: missing from {id_from} till {id_till} total {id_till - id_from} rows with time delta {time_delta} seconds')

    df.drop_duplicates(subset='id', ignore_index=True, inplace=True)
    df.timestamp = pd.to_datetime(df.timestamp, unit='ms')
    df['delta_id'] = df['id'].diff()
    df['prev_time'] = df.timestamp.shift(1)
    df.dropna(inplace=True)
    df[df.delta_id > 1].apply(count_missng, axis=1)
    del df

In [33]:
missing = []
# read_file('BTCUSDT', 'BTCUSDT-trades-2021-12-30.zip', missing)

for symbol in SYMBOLS:
    files = os.listdir(PATH + symbol)
    files.sort()
    for f in files:
        read_file(symbol, f, missing)
    # dfs = [read_file(symbol, f, missing) for f in os.listdir(PATH + symbol)]

In [2]:
def get_chunk(symbol, start):
    try:
        query = f'/fapi/v1/historicalTrades?symbol={symbol}&limit=1000&fromId={start}'
        headers= {
            'X-MBX-APIKEY': API_KEY
        }
        res = rq.get(BASE_URL + query, headers=headers)
        json = res.json()
        if res.status_code == 200:
            print(f'\r{symbol}:{start} -> {len(json)} rows', end='')
            return json
        else:
            msg = json['msg']
            code = json['code']
            print(f'\r{symbol}:{start} -> {msg} ({code})', end='')
            return None
    except Exception as e:
        print(f'\r{symbol}:{start} -> error: {e}', end='')
        return None

# %%
def get_range(symbol, fromId, tillId):
    wait_time = DEFAULT_WAIT
    err_cnt = 0
    res = []
    start = fromId
    while True:
        chunk = get_chunk(symbol, start)
        if chunk == None:
            err_cnt += 1
            wait_time = min(wait_time + 20, wait_time * 10)
        else:
            err_cnt = 0
            wait_time = DEFAULT_WAIT                
            if len(chunk) == 0: # no more data 
                print(f'\r{symbol}:{fromId} -> {len(res)} rows, no more data.')
                return res
            
            res.extend(chunk)
            # if len(res) > 99000:
            #     print(f'\r{symbol}:{fromId} -> {len(res)} rows, done 100k.')
            #     return res
            
            print(f'\r{symbol}:{fromId} -> {len(res)} rows.         ', end='')

            last = chunk[-1]
            if pd.Timestamp(last['time'], unit='ms') >= DATE_TILL:
                print(f'\r{symbol}:{fromId} -> {len(res)} rows, end date reached.')
                return res

            start = last['id'] + 1
            if start > tillId:
                print(f'\r{symbol}:{fromId} -> {len(res)} rows, tillId reached')
                return res

        if err_cnt >= 10:
            print(f'\r{symbol}: -> too many errors.    ')
            return None
    
        time.sleep(wait_time)


In [53]:
items = get_range('BTCUSDT', 839279885, 843238898)

BTCUSDT:839279885 -> 3958471 rows, tillId reached


In [60]:
df = pd.DataFrame(items)

In [61]:
df.tail()

,id,price,qty,quoteQty,time,isBuyerMaker
3958466,843239880,40490.08,0.002,80.98,1621555210216,True
3958467,843239881,40490.08,0.018,728.82,1621555210222,True
3958468,843239882,40490.08,0.035,1417.15,1621555210222,True
3958469,843239883,40490.08,0.033,1336.17,1621555210222,True
3958470,843239884,40490.08,0.335,13564.17,1621555210223,True


In [62]:
df.to_csv('../../ticks/binance/BTCUSDT/BTCUSDT-839279885-843238898.csv')

In [54]:
missing = [
    (427085758, 427134169),
    (469201388, 469234186),
    (471331093, 471378543),
    (477192555, 477325345),
    (481148451, 481707884),
    (485428888, 485665269),
    (488251057, 488924797),
    (517439800, 517446365),
    (524621349, 524621349),
    (532382597, 532642990),
    (555823694, 555823695),
    (637752062, 637883289)
]

In [65]:
for fromId, tillId in missing:
    rows = get_range('BTCUSDT', fromId, tillId)
    df = pd.DataFrame(rows)
    filename = f'../../ticks/binance/BTCUSDT/BTCUSDT-{fromId}-{tillId}.csv'
    print(f'writing to \'{filename}\'')
    df.to_csv(filename)

BTCUSDT:427085758 -> 48999 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-427085758-427134169.csv'
BTCUSDT:469201388 -> 33000 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-469201388-469234186.csv'
BTCUSDT:471331093 -> 47999 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-471331093-471378543.csv'
BTCUSDT:477192555 -> 132999 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-477192555-477325345.csv'
BTCUSDT:481148451 -> 559833 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-481148451-481707884.csv'
BTCUSDT:485428888 -> 236999 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-485428888-485665269.csv'
BTCUSDT:488251057 -> 673997 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-488251057-488924797.csv'
BTCUSDT:517439800 -> 7000 rows, tillId reached
writing to '../../ticks/binance/BTCUSDT/BTCUSDT-517439800-517446365.csv'
BTCUSDT:524621349 -> 1000 row

In [66]:
missing_eth = [
    (275173034, 275226511),
    (315571685, 315620383),
    (318209933, 318248935),
    (323290979, 323447694),
    (325981522, 326200598),
    (328435019, 328576174),
    (330510159, 330980086),
    (350130260, 350136088),
    (355354661, 355362822),
    (360542963, 360735987),
    (375620960, 375620961),
    (422257936, 422321321),
    (603994798, 607076183)
]

In [67]:
for fromId, tillId in missing_eth:
    rows = get_range('ETHUSDT', fromId, tillId)
    df = pd.DataFrame(rows)
    filename = f'../../ticks/binance/ETHUSDT/ETHUSDT-{fromId}-{tillId}.csv'
    print(f'writing to \'{filename}\'')
    df.to_csv(filename)

ETHUSDT:275173034 -> 54000 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-275173034-275226511.csv'
ETHUSDT:315571685 -> 48997 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-315571685-315620383.csv'
ETHUSDT:318209933 -> 40000 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-318209933-318248935.csv'
ETHUSDT:323290979 -> 156999 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-323290979-323447694.csv'
ETHUSDT:325981522 -> 219995 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-325981522-326200598.csv'
ETHUSDT:328435019 -> 142000 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-328435019-328576174.csv'
ETHUSDT:330510159 -> 470000 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-330510159-330980086.csv'
ETHUSDT:350130260 -> 6000 rows, tillId reached
writing to '../../ticks/binance/ETHUSDT/ETHUSDT-350130260-350136088.csv'
ETHUSDT:355354661 -> 9000 row